<center><img src=https://raw.githubusercontent.com/feast-dev/feast/master/docs/assets/feast_logo.png width=400/></center>

# Credit Risk Model Serving

### Introduction

Model serving is an exciting part of AI/ML. All of our previous work was building to this phase where we can actually serve loan predictions. 

So what role does Feast play in model serving? We've already seen that Feast can "materialize" data from the training offline store to the serving online store. This comes in handy because many models need contextual features at inference time. 

With this example, we can imagine a scenario something like this:
1. A bank customer submits a loan application on a website. 
2. The website backend requests features, supplying the customer's ID as input.
3. The backend retrieves feature data for the ID in question.
4. The backend submits the feature data to the model to obtain a prediction.
5. The backend uses the prediction to make a decision.
6. The response is recorded and made available to the user.

With online requests like this, time and resource usage often matter a lot. Feast facilitates quickly retrieving the correct feature data.

In real-life, some of the contextual feature data points could be requested from the user, and some may be retrieved from other data sources. While outside the scope of this example, Feast does facilitate retrieving request data, and joining it with other feature data. (See [Request Source](https://rtd.feast.dev/en/master/#request-source)).

In this notebook, we request feature data from the online store for a small batch of users. We then get outcome predictions from our trained model. This notebook is a continuation of the work done in the previous notebooks; it comes as the step after [03_Credit_Risk_Model_Training.ipynb](03_Credit_Risk_Model_Training.ipynb).

### Setup

*The following code assumes that you have read the example README.md file, and that you have setup an environment where the code can be run. Please make sure you have addressed the prerequisite needs.*

In [1]:
# Imports
import os
import joblib
import json
import requests
import warnings
import pandas as pd

from feast import FeatureStore, RepoConfig

In [2]:
# ingnore warnings
warnings.filterwarnings(action="ignore")

In [3]:
# Load the model
model = joblib.load("rf_model.pkl")

### Query Feast Online Server for Feature Data

We can use the Python requests library to request feature data from the online feature server (that we deployed in notebook [02_Deploying_the_Feature_Store.ipynb](02_Deploying_the_Feature_Store.ipynb)). The request takes the form of an HTTP POST command sent to the server endpoint (`url`). Recall that in the `feature_definitions.py` file, we grouped the feature views in a Feast feature service called `loan_fs`, that we can now use to request the set of features we need. The feature service and entities are supplied as part of the request's data. Finally, we also need to specify an `application/json` content type in the request header. 

In [4]:
# ID examples
ids = [18, 764, 504, 454, 453, 0, 1, 2, 3, 4, 5, 6, 7, 8]

# Submit get_online_features request to Feast online store server
response = requests.post(
    url="http://localhost:6566/get-online-features",
    # headers = {'Content-Type': 'application/x-www-form-urlencoded'},
    headers = {'Content-Type': 'application/json'},
    data=json.dumps({
        "feature_service": "loan_fs",
        "entities": {"ID": ids}
    })
)

The response is returned as JSON, with feature values for each of the IDs.

In [5]:
# Inspect the response
resp_data = json.loads(response.text)
resp_data

{'metadata': {'feature_names': ['ID',
   'credit_amount',
   'installment_commitment',
   'duration',
   'num_dependents',
   'existing_credits',
   'residence_since',
   'age']},
 'results': [{'values': [18, 764, 504, 454, 453, 0, 1, 2, 3, 4, 5, 6, 7, 8],
   'statuses': ['PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT',
    'PRESENT'],
   'event_timestamps': ['1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z',
    '1970-01-01T00:00:00Z']},
  {'values': [12579.0,
    2463.0,
    1207.0,
    4817.0,
    2670.0,
    1169.0,
    5951.0,
    2096.0,

### Get Predictions from the Model

As the response data comes in JSON format, there is a little formatting required to organize the data into a dataframe with one record per row (and features as columns).

In [6]:
# Transform JSON into dataframe
records = pd.DataFrame.from_records(
    columns=resp_data["metadata"]["feature_names"], 
    data=[[r["values"][i] for r in resp_data["results"]] for i in range(len(ids))]
)
records

,ID,credit_amount,installment_commitment,duration,num_dependents,existing_credits,residence_since,age
0,18,12579.0,4.0,24.0,1.0,1.0,2.0,44.0
1,764,2463.0,4.0,24.0,1.0,2.0,3.0,27.0
2,504,1207.0,4.0,24.0,1.0,1.0,4.0,24.0
3,454,4817.0,2.0,24.0,1.0,1.0,3.0,31.0
4,453,2670.0,4.0,24.0,1.0,1.0,4.0,35.0
5,0,1169.0,4.0,6.0,1.0,2.0,4.0,67.0
6,1,5951.0,2.0,48.0,1.0,1.0,2.0,22.0
7,2,2096.0,2.0,12.0,2.0,1.0,3.0,49.0
8,3,7882.0,2.0,42.0,2.0,1.0,4.0,45.0
9,4,4870.0,3.0,24.0,2.0,2.0,4.0,53.0


Now we can request predictions from our trained model. For convenience, we have shown the predictions along with the implied loan designations. Remember that these are predictions on loan outcomes, given context data from the loan application process.

In [7]:
# Get predictions from the model
preds = model.predict(records.loc[:, records.columns.difference(["ID"])])
pd.DataFrame({
    "ID": ids,
    "Prediction": preds,
    "Loan_Designation": ["bad" if i==0.0 else "good" for i in preds]
})

,ID,Prediction,Loan_Designation
0,18,0.0,bad
1,764,0.0,bad
2,504,0.0,bad
3,454,0.0,bad
4,453,0.0,bad
5,0,1.0,good
6,1,0.0,bad
7,2,1.0,good
8,3,0.0,bad
9,4,0.0,bad


It's important to remember that the model's predictions are like educated guesses based on learned patterns. The model will get some predictions right, and other wrong. An AI/ML team's task is generally to make the model's predictions as useful as possible in helping the organization make decisions (for example, on loan approvals).

In this case, we have a baseline model. While not ready for production, this model has set a low bar by which other models can be measured. Teams can also use a model like this to help with early testing, and with proving out things like pipelines and infrastructure before more sophisticated models are available.